In [ ]:
import os

###################################################################################
# Check that these are the correct paths running (change if necessary)

# Finn's Laptop
studydir = '/mnt/d/KPUM_NODDI/Data'
codedir = os.path.join('/home/finn/Code/KPUM_NODDI_ssd','kpum_noddi')
codedir = os.path.join('/home/finn/Code/','kpum_noddi')
softwaredir = '/home/finn/Software'

# KPUM Workstation
studydir = '/mnt/e/Finn/KPUM_NODDI/Data'
codedir = os.path.join('/home/radio/KPUM_NODDI','code/kpum_noddi')
softwaredir = os.path.join('/home/radio/KPUM_NODDI','software')

# Parallell diffusivity parameter for NODDI-model for NODDI estimation
dPar = 1.7e-3

###################################################################################
# USER INPUT - START

# Participant details
subject = '' # e.g. 035
session = '' # e.g. MR1
derivatives = '' # e.g. derivatives/dMRI_np' or 'derivatives/dMRI_op'
protocol = '' # 'NEW' or 'ORIG'

# USER INPUT - END
##################################################################################

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerfile = 'Subject_Tracker_for_dmri_dtidkinoddi_pipeline.tsv'


In [ ]:
import os

# Define I/O folders and files
#niftifolder = os.path.join(studydir, niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
datadir = os.path.join(studydir, derivatives, os.path.join(f'sub-{subject}',f'ses-{session}'))
if not os.path.exists(datadir): # then make this directory
    os.makedirs(datadir)
# TSV-file subject tracker file
subjecttrackertsv = os.path.join(studydir, derivatives, subjecttrackerfile)

def perform_process(processcall) :
    import os, subprocess
    # Perform the process given by the processcall by launching into terminal
    p=subprocess.Popen(processcall, stdout=subprocess.PIPE, shell=True)
    # Poll process.stdout to show stdout live
    while True:
        output = p.stdout.readline()
        if p.poll() is not None:
            break
        if output:
            print(output.strip().decode("utf-8"))
    rc = p.poll()


########################################
## START

import os 
import pandas as pd

currdir = os.getcwd()
os.chdir(studydir)

# Read the SubjectTracker-file.tsv
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)
# Check if dicomdatafolder has an entry in df, and if not add it to the file
if not df['participant_id'].isin([f'sub-{subject}']).any():
    # We should add as new entry in the bottom.
    new_row = {'participant_id': f'sub-{subject}', 
                'session_id': f'ses-{session}',
                'dmri_dtidkinoddi_pipeline': 'Pending',
                'QC': '',
                'comments': ''} 
    df.loc[len(df)] = new_row
    print(f"{subject} is not in {subjecttrackerfile}. Adding a it as new entry")
    # First, sort df according to 'input' column 
    df = df.sort_values(by = 'participant_id')
    # and then write to a new file
    df.to_csv(subjecttrackertsv, sep="\t", index=False)
else:
    print(f"{subject} is already in {subjecttrackerfile} ")
# Read the Subject_Tracker_dicomdir2sourcedata.tsv again (can have been updated)
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)

# Perform dmri_dtidkinoddi_pipeline if this is pending
if 'Pending' in df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}')]['dmri_dtidkinoddi_pipeline'].values :
    print(f"Start the dmri_dtidkinoddi_pipeline for {subject}")
    # Just to make sure that QC gets blank entry until we are done with the processing
    df.loc[df['participant_id'] == f'sub-{subject}', ['QC']] = ''
    
    ## Process to perform - dmri_dtidkinoddi_pipeline
    process = 'dmri_dtidkinoddi_pipeline'
    processfile = 'dmri_dtidkinoddi_pipeline.sh'
    processfilepath = os.path.join(codedir,'shell', processfile)
    processcall = f"bash {processfilepath} {subject} {session} {studydir} -derivatives {derivatives} -dPar {dPar} -p {protocol}"
    perform_process(processcall)
    
    # Update SubjectTracker-file.tsv
    df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), ['dmri_dtidkinoddi_pipeline']] = 'Done'
    df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), ['QC']] = 'Pending'
    # sort the rows according to 'input'
    df = df.sort_values(by = 'participant_id')
    # and write to subjecttrackertsv-file
    df.to_csv(subjecttrackertsv, sep="\t", index=False)
else :
    if 'Done' in df.loc[(df['participant_id'] == f'sub-{subject}') & (df['session_id'] == f'ses-{session}'), ['dmri_dtidkinoddi_pipeline']].values :
        print(f"The dmri_dtidkinoddi_pipeline is already performed for {subject}")
        print(f"If you want to re-run the dmri_dtidkinoddi_pipeline for {subject}, then changes its status to Pending in {subjecttrackerfile}")
    else :
        print(f"dmri_dtidkinoddi_pipeline is NOT marked Pending or Done for {subject}. Check entry in {subjecttrackertsv}")

os.chdir(currdir)
